# Lab 7: Sequential Network Architectures

## by Michael Doherty, Leilani Guzman, and Carson Pittman

"We've been trying to reach you about your car's extended warranty." There are few things more annoying in life than answering your phone and hearing these words. With technology being readily available to many people throughout the world, these types of scams have become much more prevalent. These "spam" messages, while not always scams, rarely add anything to society.

While advancing technology has allowed spam messages to increase in numbers over the years, it has also allowed for new ways to combat these messages. These spam detection filters can automatically block any incoming calls, emails, texts, etc. that appear to be suspicious; however, these filters need to be fairly refined, as blocking too many legitimate messages would lead to distrust in the filter's reliability.

Our dataset, titled "SMS Spam Collection Dataset", is comprised of over 5000 text messages in English, with each being tagged as either <code>spam</code> or <code>ham</code> (which means it's a legitimate message). Our task is to create a Sequential Network that can classify text messages as either <code>spam</code> or <code>ham</code>.

Link to the dataset: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

## 1. Preparation
### 1.1 Preprocessing and Tokenization
To start, we'll first read in the data.

In [47]:
import pandas as pd

df = pd.read_csv("data/spam.csv", encoding='ISO-8859-1')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


As seen above, the dataset seems to include some useless columns... Let's go ahead and remove those. We'll also rename our remaining columns so their purpose is clearer.

In [48]:
df.drop(labels=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1, inplace=True)

df.rename(columns={'v1': 'Label', 'v2': 'Text'}, inplace=True)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   Text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [58]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10)
tokenizer.fit_on_texts(df["Text"])

print("\nVocabulary:")
print(len(tokenizer.word_index))
# VOCAB_SIZE = len(tokenizer.word_index) 
# sequences = tokenizer.texts_to_sequences(df["Text"])

# padded_sequences = pad_sequences(sequences)
max_review_length = 500
X = tokenizer.texts_to_sequences(df["Text"])
X = pad_sequences(X, padding='post')
y = df["Label"]


Vocabulary:
8920


### 1.2 Performance Metric

F1 score?? We want to block as much spam as possible, but we also don't want to block legitimate texts (as that would probably be even worse than not blocking spam).

### 1.3 Training and Testing Method

StratifiedKFold, basically the same as Lab 5 (as the dataset is imbalanced).

In [119]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
import random
import numpy as np

mean_fpr = np.linspace(0, 1, 100)

def get_f1_score(X, y, new_model):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    f1_scores = []
    mean_tpr_list = []
    auc_list = []

    i = 1
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train = pad_sequences(X_train, maxlen=max_review_length)
        X_test = pad_sequences(X_test, maxlen=max_review_length)
        print('Type of X_train:', type(X_train))
        print('Type of X_test:', type(X_test))
        print('Type of y_train:', type(y_train))
        print('Type of y_test:', type(y_test))

        VOCAB_SIZE = X_train.shape[1]
        print(VOCAB_SIZE)
        history = new_model.fit(X_train, y_train, batch_size=64,
                                epochs=5, verbose=0,
                                validation_data=(X_test,y_test),
                                callbacks=[EarlyStopping(monitor='val_loss', patience=3)])
        print('after fit')
        print(f"Fold {i}")
        i += 1
        
        plt.figure(figsize=(10,4))
        plt.subplot(1,2,1)
        plt.plot(history.history['f1_score'], label='training')

        plt.ylabel('F1 Score %')
        plt.title('Training')
        plt.plot(history.history['val_f1_score'], label='validation')
        plt.title('Accuracy')
        plt.legend()

        plt.subplot(1,2,2)
        plt.plot(history.history['loss'], label='training')
        plt.ylabel('Training Loss')
        plt.xlabel('epochs')

        plt.plot(history.history['val_loss'], label='validation')
        plt.xlabel('epochs')
        plt.title('Loss')
        plt.legend()
        plt.show()

        f1_scores.append(history.history['val_f1_score'][-1].numpy())
        print('HEREEEEEEEE')
        yhat = new_model.predict(X_test, verbose=0)
        yhat = tf.round(yhat).numpy()
        print(type(yhat))

        
        fpr, tpr, thresholds = roc_curve(y_test, yhat.numpy())
        mean_tpr_list.append(np.interp(mean_fpr, fpr, tpr))
        auc_list.append(auc(fpr, tpr))

    plt.bar(range(len(f1_scores)), f1_scores)
    plt.ylim([min(f1_scores) - 0.01, max(f1_scores)])
    plt.title('Validation F1 Score')
    plt.xlabel('Fold')
    plt.ylabel('F1 Score')
    print("Average F1 Score:", np.mean(f1_scores))
    
    return f1_scores, mean_tpr_list, auc_list

## 2. Modeling

### 2.1 Model Creation

#### 2.1.1 Model 1: CNN Sequential Network

In [120]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Embedding, Input, Concatenate
from tensorflow.keras.layers import Subtract
from tensorflow.keras.utils import plot_model

NUM_CLASSES = 1
EMBED_SIZE = 50
# The input is a list of integers, 500 long
# NEED TO DEFINE X_train
sequence_input = Input(shape=(VOCAB_SIZE,), dtype='int32')

# this will reduce the input dimension from VOCAB_SIZE to 50 for each word
# the lenght will be the maximum number of words in a document, so 500
embedded_sequences = Embedding(VOCAB_SIZE,
                                EMBED_SIZE, # output dimension size
                                input_length=max_review_length)(sequence_input) # number of words in each sequence

# Starting sequence size is 500 (words) by 50 (embedded features)
x = Conv1D(64, 5, activation='relu',
           kernel_initializer='he_uniform')(embedded_sequences)

# After conv, length becomes: 500-4=496
# So overall size is 496 by 64

# Now pool across time
x = MaxPooling1D(5)(x)# after max pool, 496/5 -> 99 by 64
x = Dropout(0.2)(x)

# Extract additional features
x = Conv1D(64, 5, activation='relu',
           kernel_initializer='he_uniform')(x)

# New size is 95 after the conovlutions
x = MaxPooling1D(5)(x) # after max pool, size is 95/5 = 19 by 64
x = Dropout(0.2)(x)

# More features through CNN processing
x = Conv1D(64, 5, activation='relu',
           kernel_initializer='he_uniform')(x)
x = Dropout(0.2)(x)

# After convolution, size becomes 15 elements long
# Take the mean of these elements across features, result is 64 elements
x_mean = GlobalAveragePooling1D()(x) # this is the size to globally flatten
# Take the variance of these elements across features, result is 64 elements
x_tmp = Subtract()([x,x_mean])
x_std = GlobalAveragePooling1D()(x_tmp**2)

x = Concatenate(name='concat_1')([x_mean,x_std])

x = Dense(64, activation='relu',
          kernel_initializer='he_uniform')(x)
x = Dropout(0.2)(x)

predictions = Dense(NUM_CLASSES, activation='sigmoid',
              kernel_initializer='glorot_uniform')(x)


cnn = Model(sequence_input, predictions)

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import f1_score

cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[f1_score])

print(type(X))
y = np.array(y)
print(type(y))
f1_scores, mean_tpr_list, auc_list = get_f1_score(X, y, cnn)

In [ ]:
from sklearn import metrics as mt
from matplotlib import pyplot as plt
%matplotlib inline

# Summarize history for f1 score
plt.plot(history[0].history['f1_score'])
plt.plot(history[0].history['val_f1_score'])
plt.title('CNN f1 score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# Summarize history for loss
plt.plot(history[0].history['loss'])
plt.plot(history[0].history['val_loss'])
plt.title('CNN loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### 2.1.2 Model 2: Transformer Sequential Network

In [ ]:
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Layer


# The transformer architecture
class TransformerBlock(Layer): # inherit from Keras Layer
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.2):
        super().__init__()
        # Setup the model heads and feedforward network
        self.att = MultiHeadAttention(num_heads=num_heads,
                                      key_dim=embed_dim)
       
        # Make a two layer network that processes the attention
        self.ffn = Sequential()
        self.ffn.add( Dense(ff_dim, activation='relu') )
        self.ffn.add( Dense(embed_dim) )
       
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)


    def call(self, inputs, training):
        # Apply the layers as needed
        # Get the attention output from multi heads
        # Using same inpout here is self-attention
        # Call inputs are (query, value, key)
        # If only two inputs given, value and key are assumed the same
        attn_output = self.att(inputs, inputs)
       
        # Create residual output, with attention
        out1 = self.layernorm1(inputs + attn_output)
       
        # Apply dropout if training
        out1 = self.dropout1(out1, training=training)
       
        # Place through feed forward after layer norm
        ffn_output = self.ffn(out1)
        out2 = self.layernorm2(out1 + ffn_output)
       
        # Apply dropout if training
        out2 = self.dropout2(out2, training=training)
        #return the residual from Dense layer
        return out2

class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # create two embeddings
        # one for processing the tokens (words)
        self.token_emb = Embedding(input_dim=vocab_size,
                                   output_dim=embed_dim)
        # another embedding for processing the position
        self.pos_emb = Embedding(input_dim=maxlen,
                                 output_dim=embed_dim)


    def call(self, x):
        # create a static position measure (input)
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        # positions now goes from 0 to 500 by 1
        positions = self.pos_emb(positions)# embed these positions
        x = self.token_emb(x) # embed the tokens
        return x + positions # add embeddngs to get final embedding

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer


inputs = Input(shape=(X_train.shape[1],))
x = TokenAndPositionEmbedding(X_train.shape[1], VOCAB_SIZE, embed_dim)(inputs)
x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

x = GlobalAveragePooling1D()(x)
x = Dropout(0.2)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(NUM_CLASSES, activation='sigmoid',
              kernel_initializer='glorot_uniform')(x)


xformer = Model(inputs=inputs, outputs=outputs)
print(xformer.summary())

In [ ]:
xformer.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=[f1])

history = xformer.fit(
    X_train, y_train, batch_size=64, epochs=2,
    validation_data=(X_test, y_test)
)

In [ ]:
yhat_cnn = cnn.predict(X_test)
yhat_xformer = xformer.predict(X_test)


f1_scores = [mt.f1_score(y_test, np.round(yhat_cnn)),
       mt.f1_score(y_test, np.round(yhat_xformer))]


plt.bar([1,2],f1_scores)
plt.xticks([1,2],['CNN','XFORMER'])
plt.show()

#### 2.1.3 Model 3: Modified CNN Sequential Network

#### 2.1.4 Model 4: Modified Transformer Sequential Network

### 2.2 Adding Second Attention Layer to the Transformer

### 2.3 Model Comparsion

## 3. ConceptNet Numberbatch vs. GloVe